# Vehicle Model

In [ ]:
# Import libraries
import time
import math
import numpy as np
import socket
import struct
import sys

In [ ]:
# Define User Variables
#Global Variables
t_interval = .1  # Constant time interval [reflects CAN protocol wanted]

In [ ]:
#Import grade profile
#These are random numbers to get module working, will import real grade schedule later
g = [0, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1] #Unit is [%]
j_max = len(g)

In [ ]:
#Vehicle Model Constants
initial_kph = 55
prev_kph = initial_kph
kph = initial_kph
kph_Tx = initial_kph
mass = 11793  # Mass of vehicle [kg]
track_inertia = 1.03 * mass  # Estimated track inertia [kg]
# Constants in quadratic function for steady state road load
a = 579  # [N]
b = 0  # [N/kph]
c = 0.241512  # [N/kph^2]
ratedP = 179 # [kW]

# Using to check results via graph
graph = True
x = [0.0]
y = [initial_kph]
x_value = 0.0
y_value = initial_kph

# grade used as apart of timed drive cycle schedule
grade = 0

checks = True  # Breaks proccess if errors occur


In [ ]:
#This cell simulates the ECM module receiving APP from the CAN bus
def CAN_APP_Rx():
    """Documentation: This function reads a single CAN message if it has ArbID 0x18F00326 for Driver engine demand
    (or APP)."""
    
    global APP
    
    APP_m = 1 #Multiplier for APP from J1939 standard [counts/%]
    APP_b = -125 #Offset for APP from J1939 [%]    
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
    sock.setsockopt(socket.SOL_CAN_RAW, socket.CAN_RAW_ERR_FILTER, socket.CAN_ERR_MASK) #allows socket to receive error frames
    # The interface can be seen from the command prompt (ifconfig)
    # The can channel must be configured using the ip link commands
    interface = "vcan0"

    # Bind to the interface
    try:
        sock.bind((interface,))
    except OSError:
        print("Could not bind to interface '%s'\n" % interface)
        sys.exit()

    # To match this data structure, the following struct format can be used:
    can_frame_format = "<LB3x8s"
    # Unsigned Long Integer (little endian), unsigned char (byte), three pad bytes, eight chars (bytes)
    # Note: this is 16 bytes


    # These are defined in can.h of the Linux kernel sources
    # e.g. https://github.com/torvalds/linux/blob/master/include/uapi/linux/can.h
    print("socket.CAN_EFF_MASK = 0x{:08X}".format(socket.CAN_EFF_MASK))
    print("socket.CAN_EFF_FLAG = 0x{:08X}".format(socket.CAN_EFF_FLAG))
    print("socket.CAN_RTR_FLAG = 0x{:08X}".format(socket.CAN_RTR_FLAG))
    print("socket.CAN_ERR_FLAG = 0x{:08X}".format(socket.CAN_ERR_FLAG))

    # Enter a loop to read and display the data
    while True:
        can_packet = sock.recv(16) 
        can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)

        if (can_id == 2565866278): #ArbID for APP. The ArbID shows up as 98F00326. 18F00326 is actually sent
                                    #on the bus, not sure why it changes.
            break
    extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
    error_frame = bool(can_id & socket.CAN_ERR_FLAG)
    remote_tx_req_frame = bool(can_id & socket.CAN_RTR_FLAG)

    if error_frame:
        can_id &= socket.CAN_ERR_MASK
        can_id_string = "{:08X} (ERROR)".format(can_id)
    else: #Data Frame
        if extended_frame:
            can_id &= socket.CAN_EFF_MASK
            can_id_string = "{:08X}".format(can_id)
        else: #Standard Frame
            can_id &= socket.CAN_SFF_MASK
            can_id_string = "{:03X}".format(can_id)

    if remote_tx_req_frame:
        can_id_string = "{:08X} (RTR)".format(can_id)

    hex_data_print = ' '.join(["{:02X}".format(can_byte) for can_byte in can_data])
    print("{:12.6f} {} [{}] {}".format(time.time(), can_id_string, can_dlc, hex_data_print))
    APP = int(hex_data_print[18:20],base=16)*APP_m + APP_b
    return APP #units [%]


In [ ]:
#This cell simulates the ECM transmitting vehicle speed onto the CAN bus
def CAN_kph_Tx(kph_Tx):
    """Documentation: This function takes in the value Vehicle Speed as a floating-point number (type='float'),
    turns it into the J1939 CAN message format, and sends the message on bus 'VCAN0' using SocketCAN.
    It uses PGN 65265, SPN 84. Arbitration is set to 18. These can be changed"""
    
    speed_m = 1/256 #Multiplier for speed from J1939 standard [counts/kph]
    speed_b = 0 #Offset for speed from J1939 [kph]

    kph_counts = int(kph / speed_m)
    kph_hex = hex(kph_counts)
    kph_byte1 = int(kph_hex[2] + kph_hex[3],16)
    kph_byte2 = int(kph_hex[4] + kph_hex[5],16)

    print("kph: ",kph)
    print("kph_counts: ",kph_counts)
    print("kph_hex: ",kph_hex)
    print("kph_byte1: ",kph_byte1)
    print("kph_byte2: ",kph_byte2)
    
# Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

#Change this interface to match your desired connection
    interface = "vcan0"

# Bind to the interface
    sock.bind((interface,))

    can_frame_format = "=lB3x8s"

# Define the CAN Arbitration ID
    can_id = 0x18FEF125 #For Vehicle Speed on PGN 65265
#Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    
    can_data = struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",kph_byte2) + struct.pack("B",kph_byte1) + struct.pack("B",0xFF)
#The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
#Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
#can_packet = struct.pack(can_frame_format, can_id, can_dlc, kph_Tx)

    print(can_packet)
    
#Send out the CAN frame
    sock.send(can_packet)

    pass

In [ ]:
# Vehicle Model
# This simulates how the vehicle speed physically changes based on a given APP.
# Road-load coefficients and inertia of a Class 6 Freightliner box truck are used
# (see earlier cell)
def vehicle_model(APP, grade):
    global kph
    global prev_kph

    accel = (kph - prev_kph) / t_interval
    roadLoad_SS = a + b * kph + c * kph ** 2 
    roadLoad_inertial = accel * track_inertia / 3600
    roadLoad_grade = mass * 9.81 * math.sin(math.radians(math.atan(grade / 100)))
    roadLoad_E = roadLoad_SS + roadLoad_inertial + roadLoad_grade

    # look up tables inserted here, input APP, output tractive power)
    # lookup = pd.read_excel('LookupTable.xlsx')
    # lookup.info()

    # starting with => APP*rated power = tractive power [W] as estimate
    tractive_P = APP * ratedP / 100
    tractive_effort = tractive_P / kph * 3600
    delta_v = (tractive_effort - roadLoad_E) * t_interval / track_inertia
    kph = prev_kph + delta_v
    prev_kph = kph
    kph_Tx = kph
    return kph_Tx


In [ ]:
# Main Response Loop: Vehicle Model
CAN_kph_Tx(kph) #Sends out the first speed message to start the loop
while (checks == True):

    # Error Handlers
    #if abs(target - kph) > 30:
    #    print("ERROR: Target speed outside speed difference limit")
    #    checks = False
    #    break
        
    # Main Response Loop
    #CAN_kph_Rx()
    #cruise_controller(kph)
    #CAN_APP_Tx(APP_Tx)
    #time.sleep(0.05)
    CAN_APP_Rx()
    vehicle_model(APP,grade)
    CAN_kph_TX(kph_Tx)
    time.sleep(0.1)
    
    # Grade
    # This iterates through the grade schedule using the remainder so it can loop indefinitely
    # (i.e. driving a circular route indefinitely)
    n = j % j_max
    grade = g[n]
    j = j+1

    # Generated Graph of Response
    if graph == True:
        x_value+=t_interval
        y_value = kph
        x.append(x_value)
        y.append(y_value) 
        print(x_value)#Just put in to make sure program is running properly
        print(y_value) #Uncomment these three "print" statements to show numerical values
        #Use this stuff if running in a PC and can use matplotlib
        if x_value >= 100: #Change this number for how long you want loop to run
            graph = False
            checks = False
            #plt.plot(x,y)
            #plt.plot(x,y2)
            #plt.xlim(0,60)
            #plt.ylim(0,110)
            #plt.grid(True)
            #plt.xlabel('Time [s]')
            #plt.ylabel('Speed [kph]')
            #plt.title('Cruise Control Response')
            #plt.show()